## <span style="color:#956bbf">CDNOW Data Set & Summaries</span>
---

The file `CDNOW_master.csv` contains the entire purchase history up to the end of June 1998 of the cohort of 23,570 individuals who made their first-ever purchase at CDNOW in the first quarter of 1997. Each record in this file, 69,659 in total, comprises four fields: the **customer’s ID**, the **date of the transaction**, the **number of CDs purchased**, and the **dollar value of the transaction**. A 1/10th systematic sample of the whole cohort (2357 customers) is also available for developing models of buyer behaviour in noncontractual settings. The primary purpose of this notebook is to document the process of creating the 1/10th sample of the complete dataset. We also show how to create the summary of this dataset used in [Sales Forecast: Predicting Unit Sales Using Finite Mixture Model with Beta-Geometric Distribution](Sales-Forecast-Finite-Mixture-Beta-Geometric.ipynb), and how the full dataset is used in “RFM and CLV: Using Iso-value Curves for Customer Base Analysis".

### <span style="color:#956bbf">Imports</span>

In [1]:
import polars as pl
import numpy as np
from great_tables import GT

### <span style="color:#956bbf">Import Master Dataset</span>

In [2]:
CDNOW_master = (
    pl.scan_csv(source = 'data/CDNOW/CDNOW_master.csv', 
                has_header=False, 
                separator=',', 
                schema={'CustID': pl.Int32,     # customer id
                        'Date': pl.String,      # transaction date
                        'Quant': pl.Int16,      # number of CDs purchased
                        'Spend': pl.Float64})   # dollar value (excl. S&H)
    .with_columns(pl.col('Date').str.to_date("%Y%m%d"))
    .with_columns((pl.col('Date') - pl.date(1996,12,31)).dt.total_days().cast(pl.UInt16).alias('PurchDay'))
    .with_columns((pl.col('Spend')*100).round(0).cast(pl.Int64).alias('Spend Scaled'))
    .group_by('CustID', 'Date')
    .agg(pl.col('*').exclude('PurchDay').sum(), pl.col('PurchDay').max()) # Multiple transactions by a customer on a single day are aggregated into one
    .sort('CustID', 'Date')
    .with_columns((pl.col("CustID").cum_count().over("CustID") - 1).cast(pl.UInt16).alias("DoR"))  # DoR = Depth of Repeat ('Transaction' time: starts with 0 as trial, 1 as 1st repeat and so on)
)

display(CDNOW_master.head().collect())

CustID,Date,Quant,Spend,Spend Scaled,PurchDay,DoR
i32,date,i64,f64,i64,u16,u16
1,1997-01-01,1,11.77,1177,1,0
2,1997-01-12,6,89.0,8900,12,0
3,1997-01-02,2,20.76,2076,2,0
3,1997-03-30,2,20.76,2076,89,1
3,1997-04-02,2,19.54,1954,92,2


### <span style="color:#956bbf">Creating the 1/10th Sample</span>

Our goal is to extract a 1/10 sample from the full CDNOW database that is based on total repeat spend in the first 39 weeks, stratified by week of trial.

1) We first compute the number of repeat purchases made by each customer in the first 39 weeks (273 days) of 1997 and the dollar value of these purchases (`RptSpend`).
2) For each trial week, we sort customers (in descending order) on repeat spend in the first 39 weeks and determine the ID of every 10th customer (`sampledID`).

In [3]:
# Sampling technique - Python Method:

# RptSpend = (
#     CDNOW_master
#     .filter(pl.col('PurchDay') <= 273)
#     .with_columns(pl.when(pl.col('DoR') > 0)
#                   .then(pl.col('Spend Scaled'))
#                   .otherwise(0)
#                   .alias('Repeat Spend (Scaled)'))
#     .with_columns(pl.col('PurchDay').filter(pl.col('DoR') == 0)
#                   .first()
#                   .over('CustID')
#                   .alias('Trial Day'))
#     .group_by('CustID')
#     .agg(pl.col('Repeat Spend (Scaled)').sum(), pl.col('Trial Day').max())
#     .sort('CustID')
# )

# id_df = (
#     RptSpend.collect()
#     .with_columns(((pl.col('Trial Day') - 1) // 7 + 1).alias('Trial Week'))
#     .sort(['Trial Week','Repeat Spend (Scaled)', 'CustID'], descending=[False, True, False], maintain_order=True)
# )

# sampledID = id_df[9::10].select('CustID')

In [4]:
# MATLAB Sampling (due to numerical float precision handling differences, original sampling results cannot be replicated unless spend is scaled in MATLAB)
CDNOW_sample = (
    pl.scan_csv(source='data/CDNOW/CDNOW_sample.csv',
                has_header=False,
                separator=',',
                schema={'CustID': pl.Int32,
                        'NewID': pl.Int32,
                        'Date': pl.String,
                        'Quant': pl.Int16,
                        'Spend': pl.Float64})
    .with_columns(pl.col('Date').str.to_date("%Y%m%d"))
    .with_columns((pl.col('Date') - pl.date(1996,12,31)).dt.total_days().cast(pl.UInt16).alias('PurchDay'))
    .with_columns((pl.col('Spend')*100).round(0).cast(pl.Int64).alias('Spend Scaled'))
    .group_by('CustID', 'Date')
    .agg(pl.col('*').exclude('PurchDay').sum(), pl.col('PurchDay').max())
    .sort('CustID', 'Date')
    .with_columns((pl.col("CustID").cum_count().over("CustID") - 1).cast(pl.UInt16).alias("DoR"))      
)

### <span style="color:#956bbf">Creating Summaries of 1/10th Sample Data</span>

Creating summaries of the 1/10th sample data. We divide the 78 weeks in half: Period 1 is a 39-week calibration period while Period 2 is a 39-week longitudinal holdout used for model validation.

#### The number of repeat transactions made by each customer in each period

In [5]:
calwk = 273 # 39 week calibration period

# The number of repeat transactions made by each customer in each period
px = (
    CDNOW_sample
    .group_by('CustID', maintain_order=True)
    .agg(
        pl.col('PurchDay')
        .filter((pl.col('PurchDay') <= calwk) & (pl.col('DoR') > 0))
        .count()
        .alias('P1X'), # Period 1: Calibration Period

        pl.col('PurchDay')
        .filter((pl.col('PurchDay') > calwk) & (pl.col('DoR') > 0))
        .count()
        .alias('P2X')  # Period 2: Longitudinal Holdout Period      
    )
)

#### The number of CDs purchased and total spend across these repeat transactions

In [6]:
# The number of CDs purchased and total spend across these repeat transactions
pSpendQuant = (
    CDNOW_sample
    .join(px, on='CustID', how='left')
    .group_by('CustID', maintain_order=True)
    .agg(
        
        pl.col('Spend Scaled')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') <= pl.col('P1X')) & (pl.col('P1X') != 0))
        .sum()
        .alias('P1X Spend'),
        
        pl.col('Quant')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') <= pl.col('P1X')) & (pl.col('P1X') != 0))
        .sum()
        .alias('P1X Quant'),        
        
        pl.col('Spend Scaled')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') > pl.col('P1X')))
        .sum()
        .alias('P2X Spend'),
        
        pl.col('Quant')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') > pl.col('P1X')))
        .sum()
        .alias('P2X Quant')                
    )
)

#### The average spend per repeat transaction

In [7]:
# The average spend per repeat transaction
mx = (
    pSpendQuant
    .join(px, on='CustID', how='left')
    .with_columns(
        (pl.col('P1X Spend') / pl.col('P1X')).alias('Avg Spend per Repeat')
    ).fill_nan(0)
)

#### Recency & effective calibration period

The next step is to compute *recency*, as well as the length of time over which we have observed each customer’s purchasing behavior.

When fitting models such as the Pareto/NBD and BG/NBD to these data, we also want to know the “recency” information for each customer, as well as their effective calibration period:

For modeling purposes, “recency” is not the calendar date of the last observed purchase; rather the time origin for $t_x$ is the start of the observation period. Since we track customers’ purchasing from their first-ever purchase at CDNOW, the date of this first purchase is the time origin. Therefore, $t_x$ is the length of time between the first-ever purchase and the last observed purchase (in the calibration period), i.e., Last Purchase − First Purchase.

$T$ is the length of time we observe each customer (i.e., the time between the customer’s first-ever purchase at CDNOW and the end of the calibration period, 1997-09-
30).

In [8]:
# time of last calibration period repeat purchase (in weeks)
ttlrp = (
    CDNOW_sample
    .join(px, on='CustID', how='left')
    .with_columns(
        pl.col('PurchDay').filter(pl.col('DoR') == 0)
        .first()
        .over('CustID')
        .alias('Trial Day')
    )
    .group_by('CustID', maintain_order=True)
    .agg(
        pl.col('PurchDay', 'Trial Day')
        .filter(pl.col('DoR') <= pl.col('P1X'))
        .max()
        # .alias('LastPurch')
    )
    .with_columns(
        (pl.col('PurchDay')/7).alias('p1rec'), # Calendar week of the last observed purchase - Calendar Recency
        ((pl.col('PurchDay') - pl.col('Trial Day')) / 7).alias('t_x'), # Time to Last Repeat Purchase from Trial - Model Recency
        ((calwk - pl.col('Trial Day'))/7).alias('T') # effective calibration period (in weeks)
    )  
    .drop('PurchDay', 'Trial Day')
)

### <span style="color:#956bbf">Creating Number of Units Purchased by Week of Purchase Matrix</span>

#### What is the total number of CDs purchased each week?

In [9]:
# What is the total number of CDs purchased each week?
weeklysales = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))
    .group_by('Week')
    .agg(pl.col('Quant').sum())
    .sort('Week')
)

#### How many people made their first-ever (“trial”) purchase each week?

In [10]:
# How many people made their first-ever (“trial”) purchase each week?
weeklytriers = (
    CDNOW_master
    .filter(pl.col('DoR') == 0)
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))
    .group_by('Week')
    .agg(pl.len().alias('Incremental Triers'))
    .sort('Week')
    
)
weeklytriers.collect()

Week,Incremental Triers
u16,u32
1,1574
2,1642
3,1822
4,1924
5,2164
…,…
8,2034
9,2198
10,2165


#### What is the total number of CDs purchased by triers in their trial week?

In [11]:
# What is the total number of CDs purchased by triers in their trial week?
weeklytrierquant = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))
    .with_columns(pl.when(pl.col('DoR') == 0).then(pl.col('Week')).fill_null(strategy='forward').over('CustID').alias('Trial Week'))  
    .filter(pl.col('Trial Week') == pl.col('Week'))  # Any repeat purchasing by a customer in their trial week is added to their trial purchase
    .group_by('Week')
    .agg(pl.col('Quant').sum().alias('Triers Quant'))
    .sort('Week')
)

weeklytrierquant.collect()

Week,Triers Quant
u16,i64
1,3627
2,3630
3,4015
4,4347
5,4867
…,…
8,4501
9,4896
10,4795


#### What is the weekly total sales split between "trial" and "repeat"?

In [16]:
weeklysalessplit = (
    weeklysales
    .join(weeklytrierquant, on='Week', how='left')
    .fill_null(0)
    .with_columns(
        (pl.col('Quant') - pl.col('Triers Quant')).alias('Repeat Quant')
    )
    .rename({'Quant': 'Total', 'Triers Quant': 'Trial', 'Repeat Quant': 'Repeat'})
)

weeklysalessplit.collect()

Week,Total,Trial,Repeat
u16,i64,i64,i64
1,3627,3627,0
2,3857,3630,227
3,4512,4015,497
4,5054,4347,707
5,5843,4867,976
…,…,…,…
74,1265,0,1265
75,1383,0,1383
76,1573,0,1573


#### What is the distribution of the number of units purchased in each of the first 12 weeks?

In [13]:
# What is the distribution of the number of units purchased in each of the first 12 weeks?
dist_table = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))
    .filter(pl.col('Week') <= 12)
    .group_by('Week', 'CustID')
    .agg(pl.col('Quant').sum())    
    .sort('Week').collect()
    .pivot(on='Week', index='Quant', values='Quant', aggregate_function='len')
    .sort('Quant')
    .fill_null(0)
    .to_numpy().astype(np.int32)
)
TableOne = np.zeros((11,13), dtype=np.int32)
TableOne[1:11,0] = dist_table[0:10,0]
TableOne[1:10,1:] = dist_table[0:9,1:]
TableOne[10,1:] = np.sum(dist_table[9:,1:], axis=0)
TableOne[0,1:] = weeklytriers.collect()['Incremental Triers'].cum_sum() - np.sum(TableOne[1:,1:], axis=0)
TableOne_df = pl.from_numpy(TableOne, schema=[str(i) for i in range(13)], orient='row').with_columns(pl.col('0').cast(pl.String))

(
    GT(TableOne_df, rowname_col='0')
    .tab_header(title="Distribution of the Number of CDNOW Units Purchased", subtitle='First 12 Weeks')
    .tab_stubhead('Units Purchased')
    .fmt_integer()
    .tab_spanner(label='Week', columns=[str(i) for i in range(1, 13)])
    .opt_stylize()
)

GT(_tbl_data=shape: (11, 13)
┌─────┬─────┬──────┬──────┬───┬───────┬───────┬───────┬───────┐
│ 0   ┆ 1   ┆ 2    ┆ 3    ┆ … ┆ 9     ┆ 10    ┆ 11    ┆ 12    │
│ --- ┆ --- ┆ ---  ┆ ---  ┆   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ str ┆ i32 ┆ i32  ┆ i32  ┆   ┆ i32   ┆ i32   ┆ i32   ┆ i32   │
╞═════╪═════╪══════╪══════╪═══╪═══════╪═══════╪═══════╪═══════╡
│ 0   ┆ 0   ┆ 1478 ┆ 3033 ┆ … ┆ 14698 ┆ 16774 ┆ 18881 ┆ 20902 │
│ 1   ┆ 750 ┆ 852  ┆ 984  ┆ … ┆ 1397  ┆ 1444  ┆ 1387  ┆ 1148  │
│ 2   ┆ 383 ┆ 387  ┆ 456  ┆ … ┆ 644   ┆ 659   ┆ 677   ┆ 663   │
│ 3   ┆ 191 ┆ 214  ┆ 270  ┆ … ┆ 365   ┆ 374   ┆ 355   ┆ 367   │
│ 4   ┆ 95  ┆ 120  ┆ 114  ┆ … ┆ 179   ┆ 187   ┆ 199   ┆ 182   │
│ …   ┆ …   ┆ …    ┆ …    ┆ … ┆ …     ┆ …     ┆ …     ┆ …     │
│ 6   ┆ 36  ┆ 40   ┆ 42   ┆ … ┆ 75    ┆ 71    ┆ 72    ┆ 54    │
│ 7   ┆ 18  ┆ 12   ┆ 27   ┆ … ┆ 41    ┆ 37    ┆ 30    ┆ 43    │
│ 8   ┆ 12  ┆ 15   ┆ 9    ┆ … ┆ 23    ┆ 29    ┆ 24    ┆ 32    │
│ 9   ┆ 9   ┆ 9    ┆ 8    ┆ … ┆ 14    ┆ 9     ┆ 12    ┆ 16    │
│ 10  ┆ 25  ┆ 17   ┆ 27   ┆ … ┆ 48    ┆ 42    ┆ 50    ┆ 43    │
└─────┴─────┴──────┴──────┴───┴───────┴───────┴───────┴───────┘, _body=<great_tables._gt_data.Body object at 0x000001C33E4C8C80>, _boxhead=Boxhead([ColInfo(var='0', type=<ColInfoTypeEnum.stub: 2>, column_label='0', column_align='left', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='right', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='right', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='right', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='right', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='5', column_align='right', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='6', column_align='right', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='7', column_align='right', column_width=None), ColInfo(var='8', type=<ColInfoTypeEnum.default: 1>, column_label='8', column_align='right', column_width=None), ColInfo(var='9', type=<ColInfoTypeEnum.default: 1>, column_label='9', column_align='right', column_width=None), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10', column_align='right', column_width=None), ColInfo(var='11', type=<ColInfoTypeEnum.default: 1>, column_label='11', column_align='right', column_width=None), ColInfo(var='12', type=<ColInfoTypeEnum.default: 1>, column_label='12', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001C34494C860>, _spanners=Spanners([SpannerInfo(spanner_id='Week', spanner_level=0, spanner_label='Week', spanner_units=None, spanner_pattern=None, vars=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], built=None)]), _heading=Heading(title='Distribution of the Number of CDNOW Units Purchased', subtitle='First 12 Weeks', preheader=None), _stubhead='Units Purchased', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001C344836CF0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000001C344814EC0>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), 

In [14]:
dist_table = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))  
    .filter(pl.col('Week') <= 12)  
    .group_by('CustID', 'Week')
    .agg(pl.col('Quant').cast(pl.Int32).sum()) # Sum quantity purchased by each customer within each week 
    .group_by('Week', 'Quant')
    .agg(pl.col('CustID').len().alias('Count'))
    .sort('Quant')
    .collect()
)

dist_table_10_plus = (
    dist_table.filter(pl.col('Quant') >= 10)
    .group_by('Week')
    .agg(pl.col('Count').sum())
    .sort('Week')
    .with_columns(pl.lit(10).alias('Quant'))
    .select('Week', 'Quant', 'Count')
)

dist_table_1 = dist_table.filter(pl.col('Quant') < 10).vstack(dist_table_10_plus)

cumweeklytriers = (
    weeklytriers.collect()
    .with_columns(pl.col('Incremental Triers').cum_sum())
    .join(dist_table_1.group_by('Week').agg(pl.col('Count').sum()), on='Week', how='left')
    .with_columns((pl.col('Incremental Triers') - pl.col('Count')).alias('Count'))
    .with_columns(pl.lit(0).alias('Quant')).select('Week', 'Quant', 'Count')
)

TableOne = (
    dist_table_1.vstack(cumweeklytriers).sort('Week', 'Quant')
    .pivot(on='Week', index='Quant', values='Count')
)

(
    GT(TableOne, rowname_col='Quant')
    .tab_header(title="Distribution of the Number of CDNOW Units Purchased", subtitle='First 12 Weeks')
    .tab_stubhead('Units Purchased')
    .fmt_integer()
    .tab_spanner(label='Week', columns=[str(i) for i in range(1, 13)])
    .opt_stylize()
)

GT(_tbl_data=shape: (11, 13)
┌───────┬─────┬──────┬──────┬───┬───────┬───────┬───────┬───────┐
│ Quant ┆ 1   ┆ 2    ┆ 3    ┆ … ┆ 9     ┆ 10    ┆ 11    ┆ 12    │
│ ---   ┆ --- ┆ ---  ┆ ---  ┆   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ i32   ┆ u32 ┆ u32  ┆ u32  ┆   ┆ u32   ┆ u32   ┆ u32   ┆ u32   │
╞═══════╪═════╪══════╪══════╪═══╪═══════╪═══════╪═══════╪═══════╡
│ 0     ┆ 0   ┆ 1478 ┆ 3033 ┆ … ┆ 14698 ┆ 16774 ┆ 18881 ┆ 20902 │
│ 1     ┆ 750 ┆ 852  ┆ 984  ┆ … ┆ 1397  ┆ 1444  ┆ 1387  ┆ 1148  │
│ 2     ┆ 383 ┆ 387  ┆ 456  ┆ … ┆ 644   ┆ 659   ┆ 677   ┆ 663   │
│ 3     ┆ 191 ┆ 214  ┆ 270  ┆ … ┆ 365   ┆ 374   ┆ 355   ┆ 367   │
│ 4     ┆ 95  ┆ 120  ┆ 114  ┆ … ┆ 179   ┆ 187   ┆ 199   ┆ 182   │
│ …     ┆ …   ┆ …    ┆ …    ┆ … ┆ …     ┆ …     ┆ …     ┆ …     │
│ 6     ┆ 36  ┆ 40   ┆ 42   ┆ … ┆ 75    ┆ 71    ┆ 72    ┆ 54    │
│ 7     ┆ 18  ┆ 12   ┆ 27   ┆ … ┆ 41    ┆ 37    ┆ 30    ┆ 43    │
│ 8     ┆ 12  ┆ 15   ┆ 9    ┆ … ┆ 23    ┆ 29    ┆ 24    ┆ 32    │
│ 9     ┆ 9   ┆ 9    ┆ 8    ┆ … ┆ 14    ┆ 9     ┆ 12    ┆ 16    │
│ 10    ┆ 25  ┆ 17   ┆ 27   ┆ … ┆ 48    ┆ 42    ┆ 50    ┆ 43    │
└───────┴─────┴──────┴──────┴───┴───────┴───────┴───────┴───────┘, _body=<great_tables._gt_data.Body object at 0x000001C344F1AC90>, _boxhead=Boxhead([ColInfo(var='Quant', type=<ColInfoTypeEnum.stub: 2>, column_label='Quant', column_align='right', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='5', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='6', column_align='center', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='7', column_align='center', column_width=None), ColInfo(var='8', type=<ColInfoTypeEnum.default: 1>, column_label='8', column_align='center', column_width=None), ColInfo(var='9', type=<ColInfoTypeEnum.default: 1>, column_label='9', column_align='center', column_width=None), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10', column_align='center', column_width=None), ColInfo(var='11', type=<ColInfoTypeEnum.default: 1>, column_label='11', column_align='center', column_width=None), ColInfo(var='12', type=<ColInfoTypeEnum.default: 1>, column_label='12', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000001C344F59100>, _spanners=Spanners([SpannerInfo(spanner_id='Week', spanner_level=0, spanner_label='Week', spanner_units=None, spanner_pattern=None, vars=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], built=None)]), _heading=Heading(title='Distribution of the Number of CDNOW Units Purchased', subtitle='First 12 Weeks', preheader=None), _stubhead='Units Purchased', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001C344F1BF50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000001C33E03AA20>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(s

### <span style="color:#956bbf">RFM and CLV Data Preparation</span>

The initial exploratory analysis presented in the paper uses the full dataset (23,570 customers) excluding the purchasing data for ten buyers who purchased more than $4,000 worth of CDs across the 78-week period. Having validated the model on the 1/10 sample, the final RFM-group analysis is based on the revised “full” dataset of 23,560 customers.